# Testing different methods and parameters

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import seaborn as sns
import matplotlib.pyplot as plt
import random
from scipy.signal import find_peaks
from scipy.optimize import leastsq
from pyteomics import mass
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

## Data preprocessing

In [74]:
dataRoot = "C:/Users/Antonio/Google Drive/Orbitrap project/Data/#2-#3 small deviation file/"
peaklistRoot = "C:/Users/Antonio/Desktop/Esercizi prog/Data science project/peak list/"
data1 = [dataRoot + "/#2/1 min", dataRoot + "/#2/2 min"]
data2 = [dataRoot + "/#3/1 min", dataRoot + "/#3/2 min"]
peak_list_file = peaklistRoot + "peaklist_1e5_background.csv"



### Jokinen Peak Identification

In [27]:
def getSpectraJokinen(path):
    spectrum_data_files = []
    spectra = []

    # Get data files
    for file in [f for f in listdir(path) if isfile(join(path, f))]:
        path_to_file = join(path, file)
        #print(path_to_file)
        if file.endswith(".csv"):
            spectrum_data_files.append(path_to_file)

    # Read the spectrum files
    for file in spectrum_data_files:
        spectra.append(pd.read_csv(file))
    
    return spectra

In [40]:
spectra1 = [getSpectraJokinen(data1[0]), getSpectraJokinen(data1[1])]
spectra2 = [getSpectraJokinen(data2[0]), getSpectraJokinen(data2[1])]

peak_list = dict(pd.read_csv(peak_list_file).values)

In [20]:
def find_peak_indices(data):
    '''
    Logic is same as in Koli's implementation. Tested that produces same results with spectrums in
    "data/CI-orbi_20201117165601 folder (first smal deviation file)/1 min" folder.
    '''
    peak_start_indices = [0] #Initialize with first start index
    peak_end_indices = []
    for i, row in data.iterrows():
        if i == 0:
            continue;
        if data.iloc[i]["intensity"] == 0.0 and data.iloc[i - 1]["intensity"] == 0.0:
            peak_start_indices.append(i)
            peak_end_indices.append(i - 1)
    peak_end_indices.append(len(data) - 1) #Finalize with last end index
    return np.column_stack((peak_start_indices, peak_end_indices))

In [21]:
default_resolution = 280000

# x: data; a: height; x0: position; c: sigma or width
gauss  = lambda x, a, mu, sigma: a*np.exp(-(x-mu)**2/(2*sigma**2))

def fit_gaussian(peak, resolution=default_resolution, show=False):
    '''
    returns: a: 'height'; mu: 'position'; sigma: 'width'
    '''
    mu = np.average(peak["mz"], weights=peak["intensity"])
    sigma = mu/(resolution*2*np.sqrt(2*np.log(2)))
    errfunc  = lambda p, x, y: (y - gauss(x, p[0], p[1], sigma))
    init  = [peak["intensity"].max(), mu]
    out = leastsq(errfunc, init, args=(peak["mz"], peak["intensity"]))
    c = out[0]
    if show:
        x = np.linspace(peak["mz"].min(), peak["mz"].max(), 1000)
        fig, ax = plt.subplots(figsize=(8,6))
        ax.ticklabel_format(useOffset=False)
        plt.plot(peak["mz"], peak["intensity"], "b")
        plt.plot(x, gauss(x, c[0], c[1], sigma), "g")
        plt.plot((c[1],c[1]), (0.0,c[0]), "g", linestyle="--")
    return c[0], c[1], sigma

In [34]:
def getPeaks(spectra):
    peak_informations = []
    for spectrum in spectra:
        peak_information = []
        for start, end in find_peak_indices(spectrum):
            data = spectrum.iloc[start:(end+1)]
            max_intensity = data["intensity"].max()
            width = data["mz"].max() - data["mz"].min()
            average_mz = np.average(data["mz"], weights=data["intensity"])
            peak_information.append({
                "start": start,
                "end": end,
                "max_intensity": max_intensity,
                "average_mz": average_mz,
                "width": width
            })
        peak_informations.append(pd.DataFrame(peak_information))
    return peak_informations

In [50]:
peaks1 = [getPeaks(spectra1[0]),getPeaks(spectra1[1])]
peaks2 = [getPeaks(spectra2[0]), getPeaks(spectra2[1])]

In [56]:
def evaluatePeaks(spectra, peaks):
    # Add 'observed' column
    for spectrum, peak_information in zip(spectra, peaks):
        fitted_means = []
        for i, peak in peak_information.iterrows():
            data = spectrum.iloc[int(peak["start"]):int(peak["end"] + 1)]
            a, mu, sigma = fit_gaussian(data)
            fitted_means.append(mu)
        peak_information["observed_mz"] = fitted_means
    return peaks

In [170]:
a = evaluatePeaks(spectra1[0], peaks1[0])
b = evaluatePeaks(spectra1[0], peaks1[1])
c = evaluatePeaks(spectra2[1], peaks2[0])
d = evaluatePeaks(spectra2[1], peaks2[1])
ePeaks1 = [a,b]
ePeaks2 = [c,d]

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
def identify_peaks(peak_information, peak_list, theta = 0.01):
    peak_information["formula"] = None # For strings
    peak_information["formula_mz"] = np.nan # For floats
    for key in peak_list.keys():
        formula = key
        true_mz = peak_list[key]
        closest = peak_information[np.abs(peak_information["observed_mz"] - true_mz) < theta]
        if len(closest) > 0:
            closest = closest.iloc[(closest['observed_mz'] - true_mz).abs().argsort()[:1]]
            #print("True: {}; Observed: {}; Index: {}".format(true_mz, closest["observed"].values[0], closest.index.values[0]))
            peak_information.at[closest.index.values[0], "formula"] = formula
            peak_information.at[closest.index.values[0], "formula_mz"] = true_mz
    peak_information["error"] =  peak_information["formula_mz"] - peak_information["observed_mz"]

In [ ]:
completePeaks1 = [identify_peaks(ePeaks1[0], peak_list),identify_peaks(ePeaks1[1], peak_list)]
completePeaks2 = [identify_peaks(ePeaks2[0], peak_list),identify_peaks(ePeaks2[1], peak_list)]

### Koli Peak Identification

In [138]:
import numpy as np

# Find indices of the different peaks in the data
def findPeakIndices(data):
    n = int(round(sum(data[:,1] == 0)/2)+1)
    indices = np.zeros([n,2], dtype="uint32")
    a = False
    ii = 0
    for i in range(len(data[:,1])-1):
        if data[i,1] == 0 and a:
            indices[ii, 1] = i - 1
            if ii != n:
                indices[ii+1, 0] = i
            ii += 1
        a = data[i,1] == 0
    last = n-sum(indices[:,0]>=indices[:,1])
    indices = indices[0:last+1,:]
    indices[last, 1] = len(data[:,0]) - 1
    return indices

In [68]:
from scipy.optimize import leastsq
import numpy as np

# Get one mean by fitting a Gaussian distribution to the data
# with resolution of 280000
def getMean(peak):
    mu = np.average(peak[:,0], weights=peak[:,1])
    sigma = mu/(280000*2*np.sqrt(2*np.log(2)))
    fitfunc  = lambda p, x: p[0]*np.exp(-0.5*((x-p[1])/sigma)**2)
    errfunc  = lambda p, x, y: (y - fitfunc(p, x))
    init  = [1.0, mu]
    out = leastsq(errfunc, init, args=(peak[:,0], peak[:,1]))
    c = out[0]
    return c

In [69]:
from scipy.optimize import minimize
import numpy as np
import scipy.signal as scs

# Calculate mean square error for the model
def MSE(x, y, a, mu, sigma):
    norm = lambda a, mu, x: a*np.exp(-0.5*((x-mu)/sigma)**2)
    error = 0
    for i in range(len(y)):
        error = error + (y[i] - np.sum(norm(a,mu,x[i])))**2
    return error/len(y)

def errorFunction(x, y, sigma):
    return lambda params: MSE(x, y, params[::2], params[1::2], sigma)

# Function used to group values that are too close (abs(mu[n]-mu[n+1])<th) to each other together
def group(mu, th):
    groups = []
    unassigned = np.linspace(0,len(mu)-1,len(mu)).astype(int)
    while len(unassigned) > 0:
        dist = abs(np.array(mu[unassigned]) - mu[unassigned[0]])
        group = unassigned[dist < th]
        unassigned = unassigned[dist >= th]
        groups += [group]
    return groups



# Get one or more means depending on if there are multiple peaks in the data
# by fitting one or more Gaussian distributions to the data using resolution of 280000 
def getMeans(peaks):
    ind = scs.find_peaks(peaks[:,1], max(peaks[:,1])/20, prominence=max(peaks[:,1])/10)[0]
    mu = peaks[ind,0]
    n = len(mu)
    sigma = mu/(280000*2*np.sqrt(2*np.log(2)))
    
    init  = []
    for i in range(len(mu)):
        init += [np.mean(peaks[:,1]), mu[i]]
    
    # Fit n Gaussian distributions to the data
    f = errorFunction(peaks[:,0], peaks[:,1], sigma)
    params = minimize(f, init, method='BFGS').x
    
    a = params[::2]
    mu = params[1::2]
    
    # Group fitted distributions that are too close to each other together
    # and combine them
    groups = group(mu, 1e-5)
    a2 = np.zeros(len(groups))
    mu2 = np.zeros(len(groups))
    sigma2 = np.zeros(len(groups))
    for i in range(len(groups)):
        mu2[i] = np.mean(mu[groups[i]])
        a2[i] = np.sum(a[groups[i]])
        sigma2[i] = np.mean(sigma[groups[i]])
    return mu2, a2, sigma2

In [70]:
import numpy as np

# Get all peak means for the data
def getAllMeans(peaks):
    indices = findPeakIndices(peaks)
    n = len(indices[:,0])
    means = []
    aa = []
    sigmas = []
    peaki = []
    for i in range(n):
        peak = peaks[indices[i,0]:indices[i,1]+1,:]
        mu, a, sigma = getMeans(peak)
        for ii in range(len(a)):
            means += [mu[ii]]
            aa += [a[ii]]
            sigmas += [sigma[ii]]
            peaki += [i]
    return means, aa, sigmas, peaki

In [91]:
import pandas as pd
import numpy as np

# Identify peaks and return a data frame of peaks and their properties
def identifyPeaks(data, peaklist, th):
    peaklist['observed'] = np.zeros(len(peaklist['mz']))
    peaklist['a'] = np.zeros(len(peaklist['mz']))
    peaklist['sigma'] = np.zeros(len(peaklist['mz']))
    peaklist['peak'] = np.zeros(len(peaklist['mz']))
    peakMeans, a, sigmas, ind = getAllMeans(data)
    trueValue = peaklist['mz'].to_numpy()
    for i in range(len(trueValue)):
        if min(abs(peakMeans - trueValue[i])) < th:
            j = np.argmin(abs(peakMeans - trueValue[i]))
            peaklist.iloc[i,2] = peakMeans[j]
            peaklist.iloc[i,3] = a[j]
            peaklist.iloc[i,4] = sigmas[j]
            peaklist.iloc[i,5] = ind[j]
    unidentified = np.linspace(0, len(trueValue)-1, len(trueValue))
    unidentified = unidentified[peaklist['observed'].to_numpy() == 0]
    return peaklist.drop(unidentified)

In [135]:
import numpy as np
import pandas as pd

def readFile(fileName):
    return pd.read_csv(fileName).iloc[1:,:2].to_numpy().astype("float64")

def getSpectraKoli(path):
    spectrum_data_files = []
    spectra = []

    # Get data files
    for file in [f for f in listdir(path) if isfile(join(path, f))]:
        path_to_file = join(path, file)
        #print(path_to_file)
        if file.endswith(".csv"):
            spectrum_data_files.append(path_to_file)

    # Read the spectrum files
    for file in spectrum_data_files:
        spectra.append(readFile(file))
    
    return spectra

In [136]:
spectra1 = [getSpectraKoli(data1[0]), getSpectraKoli(data1[1])]
spectra2 = [getSpectraKoli(data2[0]), getSpectraKoli(data2[1])]

In [143]:
peaklist = pd.read_csv(peak_list_file)

a = []
for s in spectra1[0]:
    a.append(identifyPeaks(s, peaklist, 1e-4))

b = []
for s in spectra1[1]:
    b.append(identifyPeaks(s, peaklist, 1e-4))
    
completePeaks1 = [a,b]
    
c = []
for s in spectra2[0]:
    c.append(identifyPeaks(s, peaklist, 1e-4))
d = []
for s in spectra2[1]:
    d.append(identifyPeaks(s, peaklist, 1e-4))
completePeaks2 = [c,d]

C:\Users\Antonio\anaconda3\envs\orbitool\lib\site-packages\scipy\optimize\optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
C:\Users\Antonio\anaconda3\envs\orbitool\lib\site-packages\scipy\optimize\optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
C:\Users\Antonio\anaconda3\envs\orbitool\lib\site-packages\scipy\optimize\optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
C:\Users\Antonio\anaconda3\envs\orbitool\lib\site-packages\scipy\optimize\optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
C:\Users\Antonio\anaconda3\envs\orbitool\lib\site-packages\scipy\optimize\optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
C:\Users\Antonio\anaconda3\envs\orbitool\lib\site-packages\scipy\optimize\o

In [154]:
for d in completePeaks1:
    for s in d:
        s["error"] =  s["mz"] - peak_information["observed"]
for d in completePeaks2:
    for s in d:
        s["error"] =  s["mz"] - peak_information["observed"]

## Selection method 1

In [144]:
def selectBestPeaks(data, n=10, threshold=0, forceThreshold=False, test_n=0, randomTestSamples=True, debug=False):
    if n <=0:
        return None
    result = data.copy()
    result["absolute_error"] =  abs(data["error"])
    result = result.sort_values(by=['absolute_error'], ascending=True) #specified ascending parameter for flexibility
    diff = result
    test_n = abs(test_n)
    
    if debug:    
        print(result)
    
    if threshold:
        threshold = abs(threshold)
        filtered = result[result['absolute_error']<=threshold]
    
    if len(result) > n:
        if threshold:
            if len(filtered) > n:
                result = filtered[0:n]
            else:
                if forceThreshold:
                    result = filtered
                else:
                    result = result[0:n]
        else:
            result = result[0:n]
    else:
        if forceThreshold and threshold:
            result = filtered
        else:
            x = max(len(result), n)
            result = result[0:x]
            
    diff = pd.concat([result,diff]).drop_duplicates(keep=False)
    if test_n != 0:
        test_n = min(test_n, len(diff))
        if randomTestSamples:
            diff = diff.sample(test_n)
        else:
            diff = diff[0:test_n]
    return result, diff

## Selection method 2

In [145]:
# Choose n peaks with the smallest error
def selectPeaksMinimumError(peaks, n):
    errors = abs(peaks.iloc[:,1]-peaks.iloc[:,2])
    ind = np.argpartition(errors, n)
    return peaks.iloc[ind[:n],:], peaks.iloc[ind[n:],:]

# Choose n peaks with error closest to the mean error
def selectPeaksMeanError(peaks, n):
    errors = abs(peaks.iloc[:,1]-peaks.iloc[:,2])
    ind = np.argpartition(abs(errors-np.mean(errors)), n)
    return peaks.iloc[ind[:n],:], peaks.iloc[ind[n:],:]

# Choose n peaks with error closest to the median error
def selectPeaksMedianError(peaks, n):
    errors = abs(peaks.iloc[:,1]-peaks.iloc[:,2])
    ind = np.argpartition(abs(errors-np.median(errors)), n)
    return peaks.iloc[ind[:n],:], peaks.iloc[ind[n:],:]

# Splits the peaks into k partitions and selects n peaks from each partition
# so it results in n*k peaks for training and the rest of the peaks for testing
def selectPeaks(peaklist, n, k, select=selectPeaksMinimumError):
    n_peaks = len(peaklist.iloc[:,0])
    partition_size = int((n_peaks - (n_peaks % k)) / k)
    test = pd.DataFrame(columns=["formula","mz","observed","a","sigma","peak"])
    train = pd.DataFrame(columns=["formula","mz","observed","a","sigma","peak"])
    for i in range(k):
        if i < k-1:
            partition = peaklist.iloc[i*partition_size:(i+1)*partition_size,:]
        else:
            partition = peaklist.iloc[i*partition_size:,:]
        tr, ts = select(partition, n)
        test = test.append(ts)
        train = train.append(tr)
    return train, test

In [146]:
def combineObservations(peaklists):
    peaklist = peaklists[0][["formula", "mz", "observed"]].set_index("formula")
    for i in range(len(peaklists)-1):
        peaklist = peaklist.join(peaklists[i+1][["formula", "observed"]].set_index("formula"), on="formula", rsuffix='_'+str(i+1))
    return peaklist

def combinePeaklists(peaklists):
    peaklist = peaklists[0].set_index("formula")
    for i in range(len(peaklists)-1):
        peaklist = peaklist.join(peaklists[i+1].drop("mz", axis=1).set_index("formula"), on="formula", rsuffix='_'+str(i+1))
    return peaklist

## Model

In [147]:
def fit_poly_reg_model(x, y, degree, show = False, ax = None, color="r"):
    poly_features = PolynomialFeatures(degree)
    model = make_pipeline(poly_features, LinearRegression()).fit(x.reshape(-1, 1), y)
    if show:
        print("Coefficients: {}".format(model.steps[1][1].coef_[1:]), "Independent: {}".format(model.steps[1][1].intercept_))
        preds = model.predict(x.reshape(-1, 1))
        sns.set_theme(style="whitegrid")
        if not ax:
            fig, ax = plt.subplots(figsize=(8,6))
        sns.scatterplot(x=x, y=y, ax=ax, color=color)
        plt.plot(x, preds, linestyle="-", color=color)
    return model

## Testing

In [161]:
train_information = []
for n in [10, 20, 30, 40, 50]:
    for degree in [1, 2,3,4,5]:
        mses = []
        for peak_information in completePeaks1[0]:
            peak_information = peak_information[peak_information["error"].notnull()]
            n_data = len(peak_information)
            train, test = selectBestPeaks(peak_information, n=n, test_n=15)
            if len(test) > 0:
                model = fit_poly_reg_model(train["observed"].values, train["error"].values*-1, degree)
                preds = model.predict(test["observed"].values.reshape(-1, 1))
                mse = mean_squared_error(test["error"].values*-1, preds)
                mses.append(mse)
        train_information.append({
            "degree": degree,
            "n": n,
            "mean_mse": np.mean(mses),
            "min_mse": np.min(mses),
            "max_mse": np.max(mses),
            "std_mse": np.std(mses)
        })
pd.DataFrame(train_information).sort_values(by=['mean_mse', "std_mse"], ascending=True)

,degree,n,mean_mse,min_mse,max_mse,std_mse
1,2,10,1.853211e-09,7.819238e-10,3.149571e-09,5.477661e-10
0,1,10,1.936348e-09,6.966510e-10,3.959344e-09,6.477692e-10
2,3,10,1.947731e-09,7.650682e-10,3.482009e-09,6.044798e-10
6,2,20,2.246044e-09,1.041990e-09,3.884852e-09,7.021447e-10
5,1,20,2.253383e-09,1.173759e-09,3.648695e-09,5.801341e-10
12,3,30,2.386982e-09,1.239207e-09,3.702707e-09,6.155430e-10
7,3,20,2.399587e-09,7.500274e-10,4.350686e-09,6.530806e-10
10,1,30,2.473240e-09,1.103657e-09,4.889916e-09,6.680299e-10
3,4,10,2.504485e-09,9.180536e-10,6.258221e-09,1.095243e-09
8,4,20,2.505554e-09,1.097160e-09,4.774687e-09,7.258680e-10


In [169]:
train_information = []
for part in range(2, 5):
    for ppp in  range(2, 5):
        for degree in [1, 2,3,4,5]:
            mses = []
            for peak_information in completePeaks1[0]:
                peak_information = peak_information[peak_information["error"].notnull()]
                n_data = len(peak_information)
                train, test = selectPeaks(peak_information, ppp, part)
                if len(test) > 0:
                    model = fit_poly_reg_model(train["observed"].values, train["error"].values*-1, degree)
                    preds = model.predict(test["observed"].values.reshape(-1, 1))
                    mse = mean_squared_error(test["error"].values*-1, preds)
                    mses.append(mse)
            train_information.append({
                "degree": degree,
                "partitions": part,
                "n": ppp,
                "mean_mse": np.mean(mses),
                "min_mse": np.min(mses),
                "max_mse": np.max(mses),
                "std_mse": np.std(mses)
            })
pd.DataFrame(train_information).sort_values(by=['mean_mse', "std_mse"], ascending=True)

,degree,partitions,n,mean_mse,min_mse,max_mse,std_mse
20,1,4,3,1.703651e-09,9.649575e-10,3.219732e-09,4.136375e-10
10,1,3,4,1.710236e-09,9.194992e-10,2.952355e-09,4.017287e-10
15,1,4,2,1.739201e-09,9.884668e-10,3.200691e-09,4.479222e-10
25,1,4,4,1.749500e-09,1.042856e-09,2.779308e-09,3.485614e-10
5,1,3,3,1.778744e-09,8.510721e-10,3.568111e-09,5.189035e-10
0,1,3,2,1.813760e-09,1.066527e-09,3.570747e-09,5.731101e-10
26,2,4,4,1.892372e-09,1.173168e-09,2.996304e-09,4.534042e-10
11,2,3,4,1.918126e-09,1.139422e-09,3.518186e-09,5.008936e-10
21,2,4,3,1.959023e-09,1.130364e-09,3.725574e-09,5.623201e-10
6,2,3,3,2.176623e-09,8.945033e-10,4.685878e-09,7.557516e-10
